In [ ]:
import os
import pandas as pd
import usaddress

In [ ]:
def getaddress(addr):
    if addr == 'No Address Supplied':
        return 'No Address Supplied', '', '', '', ''
    else:
        try:
            dictionary = usaddress.tag(addr, tag_mapping={
                'Recipient': 'recipient',
                'AddressNumber': 'address1',
                'AddressNumberPrefix': 'address1',
                'AddressNumberSuffix': 'address1',
                'StreetName': 'address1',
                'StreetNamePreDirectional': 'address1',
                'StreetNamePreModifier': 'address1',
                'StreetNamePreType': 'address1',
                'StreetNamePostDirectional': 'address1',
                'StreetNamePostModifier': 'address1',
                'StreetNamePostType': 'address1',
                'CornerOf': 'address1',
                'IntersectionSeparator': 'address1',
                'LandmarkName': 'address1',
                'USPSBoxGroupID': 'address1',
                'USPSBoxGroupType': 'address1',
                'USPSBoxID': 'address1',
                'USPSBoxType': 'address1',
                'BuildingName': 'address2',
                'OccupancyType': 'address2',
                'OccupancyIdentifier': 'address2',
                'SubaddressIdentifier': 'address2',
                'SubaddressType': 'address2',
                'PlaceName': 'city',
                'StateName': 'state',
                'ZipCode': 'zip_code',
            })[0]
        except:
            print("error parsing address: {}".format(addr))
            dictionary = {"address1":addr}
        try:
            add1 = dictionary['address1']
        except:
            add1 = addr
        try:
            add2 = dictionary['address2']
        except:
            add2 = ''
        try:
            city = dictionary['city']
        except:
            city = ''
        try:
            state = dictionary['state']
        except:
            state = ''
        try:
            zip = dictionary['zip_code']
        except:
            zip = ''

        return add1, add2, city, state, zip

def remove_pref(cell):
    try:
        if isinstance(cell, float):
            return 'No Address Supplied'
        else:
            new_cell = cell.split('Preferred')[0].rstrip()
            return new_cell
    except:
        return cell

def isanonymous(first, last):
    if isinstance(first, str) and isinstance(last, str):
        return 'N'
    else:
        return 'Y'

def anon(name):
    if isinstance(name, str):
        return name
    else:
        return 'Anon'

In [ ]:
p = os.getcwd()
comment_file = os.path.join(p, 'TPCM_Comments.xls')

comments = pd.read_excel(comment_file)
first_name = comments['First Name (optional)']
last_name = comments['Last Name (optional)']
email = comments['I prefer to be contacted at this email address (optional)']
phone = comments['I prefer to be contacted at this phone number (optional)']
date = comments['CreationDate']
comment = comments['Comment']
full_address = comments['I prefer to be contacted at this mailing address (optional)']

frames = [first_name, last_name, full_address, email, phone, date, comment]
df = pd.concat(frames, axis=1)
df = df.rename(columns={"First Name (optional)": "FirstName", 
                   "Last Name (optional)": "LastName", 
                   "I prefer to be contacted at this mailing address (optional)": "FullAddress",
                   "I prefer to be contacted at this email address (optional)": "Email",
                   "I prefer to be contacted at this phone number (optional)": "Phone",
                   "CreationDate": "DateSubmitted",
                   "Comment": "Letter Text"
                  })
new_addr = df['FullAddress'].apply(remove_pref)
new_addr = new_addr.to_frame()
new_addr = new_addr.rename(columns = {"FullAddress": "NewAddress"})                         
frames = [df, new_addr]
df = pd.concat(frames, axis=1)

# df['DateSubmitted'] = df['DateSubmitted']
df['DateSubmitted'] = pd.to_datetime(df['DateSubmitted']).dt.date
print("If you get errors here you can look back at the address and try to fix them (the parser does not like #s and repeated values)")
print("Parssing out Address 1:\n")
df["Address1"] = df.apply(lambda row: getaddress(str(row["NewAddress"]))[0], axis=1)
print("Parssing out Address 2:\n")
df["Address2"] = df.apply(lambda row: getaddress(str(row["NewAddress"]))[1], axis=1)
print("Parssing out City:\n")
df["City"] = df.apply(lambda row: getaddress(str(row["NewAddress"]))[2], axis=1)
print("Parssing out State:\n")
df["State"] = df.apply(lambda row: getaddress(str(row["NewAddress"]))[3], axis=1)
print("Parssing out ZipPostal:\n")
df["ZipPostal"] = df.apply(lambda row: getaddress(str(row["NewAddress"]))[4], axis=1)
df['Anonymous'] = 'N'
df['Delivery Type'] = '9'
df['WithinCommentPeriod'] = '1'
df['Common Interest'] = '3'
df['ContactMethod'] = 'Email'       
df['Organization'] = ''
df['Title'] = ''
df['Country'] = ''
df['Province/Region'] = ''
df['Anonymous'] = df.apply(lambda x: isanonymous(x['FirstName'], x['LastName']), axis=1)
df['FirstName'] = df.apply(lambda x: anon(x['FirstName']), axis=1)
df['LastName'] = df.apply(lambda x: anon(x['LastName']), axis=1)
df = df.drop(['FullAddress', 'NewAddress'], axis=1)
df = df[['FirstName', 'LastName', 'Anonymous', 'Organization', 'Title', 
         'Address1', 'Address2', 'City', 'State', 'Province/Region', 'ZipPostal',
         'Country', 'Email', 'Phone', 'ContactMethod', 'Letter Text', 'DateSubmitted',
         'Delivery Type', 'WithinCommentPeriod', 'Common Interest'
        ]]
print("\nCompleted successfully...")

In [ ]:
df.to_csv(os.path.join(p, "TPCM_Data_to_CARA.csv"), index=False)